In [10]:
import configparser
import logging
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR
from pathlib import Path
from model import DnCNN
import data_generator as dg
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import random
from timeit import default_timer as timer
import cv2
from skimage.metrics import peak_signal_noise_ratio
from skimage.metrics import structural_similarity
import glob
from noise_model import poissonpoissonnoise as nm
import datetime
from vgg import Vgg16
from torchvision import transforms
import torchvision

In [11]:
test_data_dir = "/projectnb/ec523/mcokbas/Final_project/KAIR/testsets/set12"
eta_min = 2
eta_max = 8
dose = 0.2

test_file_list = glob.glob(test_data_dir + "/*.png")
xs_test = []
# Can't directly convert the xs_test from list to ndarray because some images are 512*512
# while the rest are 256*256.
for i in range(len(test_file_list)):
    img = cv2.imread(test_file_list[i], 0)
    img = np.array(img, dtype="float32") / 255.0
    img = np.expand_dims(img, axis=0)
    img_noisy, _ = nm(img, eta_min, eta_max, dose, t=100)
    xs_test.append((img_noisy, img))

In [15]:
model_save_dir = "/projectnb/ec523/mcokbas/Final_project/DnCNN_poisson_perceptual/model_path/dose20"
model_path = Path(model_save_dir) / "{}.pth".format(20)
net = DnCNN(depth=17, n_channels=64,
                  img_channel=1,
                  use_bnorm= 'on',
                  kernel_size=3)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net = net.to(device)
# Restore the model.
net.load_state_dict(torch.load(model_path))

net.eval()


with torch.no_grad():
    for idx, test_data in enumerate(xs_test):
        inputs, labels = test_data
        inputs = np.expand_dims(inputs, axis=0)
        inputs = torch.from_numpy(inputs).to(device)
        labels = labels.squeeze()

        outputs = net(inputs)
        saving_img_dir = "/projectnb/ec523/mcokbas/Final_project/DnCNN_poisson_perceptual/test_res_imgs/"+ str(idx) + ".png" 
        saving_orig_dir = "/projectnb/ec523/mcokbas/Final_project/DnCNN_poisson_perceptual/test_input_imgs/"+ str(idx) + ".png" 
        torchvision.utils.save_image(outputs, saving_img_dir)
        torchvision.utils.save_image(inputs, saving_orig_dir)
        print(type(outputs))
#         outputs = outputs.squeeze().cpu().detach().numpy()

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>


In [7]:
img = xs_test[0]
print(type(img))

<class 'tuple'>
